In [1]:
import tensorflow as tf

In [2]:
# force CPU (make CPU visible)
cpus = tf.config.experimental.list_physical_devices('CPU')
print(cpus)
tf.config.set_visible_devices([], 'GPU')  # hide the GPU
tf.config.set_visible_devices(cpus[0], 'CPU') # unhide potentially hidden CPU
tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

## Tensores

In [10]:
rank_0_tensor = tf.constant(4)
print(rank_0_tensor)

tf.Tensor(4, shape=(), dtype=int32)


In [11]:
rank_1_tensor = tf.constant([1,2,3])
print(rank_1_tensor)

tf.Tensor([1 2 3], shape=(3,), dtype=int32)


In [16]:
rank_2_tensor = tf.constant([[1.,2.],[3.,4.]])
print(rank_2_tensor)

tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float32)


In [17]:
rank_3_tensor = tf.constant([[[1,2],[3,4]],[[5,6],[7,8]]], dtype=tf.float16)
print(rank_3_tensor)

tf.Tensor(
[[[1. 2.]
  [3. 4.]]

 [[5. 6.]
  [7. 8.]]], shape=(2, 2, 2), dtype=float16)


In [5]:
a = [[2.]]
b = [[5.]]
m = tf.matmul(a, b)
print("{} * {} = {}".format(a,b,m))
m

[[2.0]] * [[5.0]] = [[10.]]


<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[10.]], dtype=float32)>

In [24]:
tf.reshape(rank_3_tensor, (8,))

<tf.Tensor: shape=(8,), dtype=float16, numpy=array([1., 2., 3., 4., 5., 6., 7., 8.], dtype=float16)>

In [25]:
tf.reshape(rank_3_tensor, (8,1))

<tf.Tensor: shape=(8, 1), dtype=float16, numpy=
array([[1.],
       [2.],
       [3.],
       [4.],
       [5.],
       [6.],
       [7.],
       [8.]], dtype=float16)>

## Variáveis

https://www.tensorflow.org/guide/variable

In [18]:
my_tensor = tf.constant([[1.0, 2.0], [3.0, 4.0]])
my_variable = tf.Variable(my_tensor)

# Variables can be all kinds of types, just like tensors
bool_variable = tf.Variable([False, False, False, True])
complex_variable = tf.Variable([5 + 4j, 6 + 1j])

In [19]:
print("Shape: ", my_variable.shape)
print("DType: ", my_variable.dtype)
print("As NumPy: ", my_variable.numpy())

Shape:  (2, 2)
DType:  <dtype: 'float32'>
As NumPy:  [[1. 2.]
 [3. 4.]]


In [20]:
print("A variable:", my_variable)
print("\nViewed as a tensor:", tf.convert_to_tensor(my_variable))
print("\nIndex of highest value:", tf.argmax(my_variable))

# This creates a new tensor; it does not reshape the variable.
print("\nCopying and reshaping: ", tf.reshape(my_variable, [1,4]))

A variable: <tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[1., 2.],
       [3., 4.]], dtype=float32)>

Viewed as a tensor: tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float32)

Index of highest value: tf.Tensor([1 1], shape=(2,), dtype=int64)

Copying and reshaping:  tf.Tensor([[1. 2. 3. 4.]], shape=(1, 4), dtype=float32)


In [36]:
# Create a and b; they will have the same name but will be backed by
# different tensors.
a = tf.Variable(my_tensor, name="Mark")
# A new variable with the same name, but different value
# Note that the scalar add is broadcast
b = tf.Variable(my_tensor + 1, name="Mark")

# These are elementwise-unequal, despite having the same name
print(a == b)

tf.Tensor(
[[False False]
 [False False]], shape=(2, 2), dtype=bool)


In [35]:
step_counter = tf.Variable(1, trainable=False)

## Diferenciação automática

https://www.tensorflow.org/guide/autodiff

In [6]:
x = tf.Variable(3.0)

with tf.GradientTape() as tape:
  y = x**2

In [7]:
# dy = 2x * dx
dy_dx = tape.gradient(y, x)
dy_dx.numpy()

6.0

In [10]:
with tf.GradientTape() as tape:
    y = x**2
    x.assign(2.0)

In [11]:
# dy = 2x * dx
dy_dx = tape.gradient(y, x)
dy_dx.numpy()

4.0

## Gráfico e modelos

In [12]:
def simple_relu(x):
  if tf.greater(x, 0):
    return x
  else:
    return 0

# `tf_simple_relu` is a TensorFlow `Function` that wraps `simple_relu`.
tf_simple_relu = tf.function(simple_relu)

print("First branch, with graph:", tf_simple_relu(tf.constant(1)).numpy())
print("Second branch, with graph:", tf_simple_relu(tf.constant(-1)).numpy())

First branch, with graph: 1
Second branch, with graph: 0


In [13]:
class SimpleModule(tf.Module):
    
  def __init__(self, name=None):
    
    super().__init__(name=name)
    
    self.a_variable = tf.Variable(5.0, name="train_me")
    
    self.non_trainable_variable = tf.Variable(5.0, trainable=False, name="do_not_train_me")
    
  def __call__(self, x):
    
    return self.a_variable * x + self.non_trainable_variable

simple_module = SimpleModule(name="simple")

simple_module(tf.constant(5.0))

<tf.Tensor: shape=(), dtype=float32, numpy=30.0>

In [14]:
# All trainable variables
print("trainable variables:", simple_module.trainable_variables)
# Every variable
print("all variables:", simple_module.variables)

trainable variables: (<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>,)
all variables: (<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>, <tf.Variable 'do_not_train_me:0' shape=() dtype=float32, numpy=5.0>)
